# Retreiving and Preparing Text for Machines

In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib 
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

from IPython.display import display

### Not working

In [2]:
india_search = 'http://infoweb.newsbank.com.proxy.uchicago.edu/resources/search/nb?p=WORLDNEWS&t=continent%3AAsia%21Asia%2Fcountry%3AIndia%21India%2Fpubname%3ABTI2%21Times%2Bof%2BIndia%252C%2BThe%2B%2528Mumbai%252C%2BIndia%2529%2Fyear%3A2017%212017&fld0=alltext&val0=%22climate+change%22&bln1=AND&fld1=YMD_date&val1=&sort=_rank_%3AD'

In [3]:
requests.get(india_search)

<Response [200]>

In [4]:
indiaContentRequest = requests.get(india_search)
#print(indiaContentRequest.text)

### Scraping Text (from a saved .html file)

In [93]:
countryDict = {'UK': ["climateUKGuard.html"], 'Africa': ["climateAfrica.html"], 'India': ["climateIndia.html"]}

In [95]:
def loopThruArticles(countryDict):

    parsDict = {'region': [], 'source': [], 'title': [], 'text': [], 'date': [], 'author': [], 'section': []}

    for key, val in countryDict.items():
        for html in val:
            parsDict['region'].append(key)
            parseArticle(html, parsDict)
    
    return parsDict
    

In [104]:
def parseArticle(html, parsDict):
    soup = bs4.BeautifulSoup(open(html, 'rt', encoding='UTF8'))

    # title
    title0 = soup.find_all('div', {'class' : 'title'})
    title1 = title0[0].text.replace('Hide Details', '')
    title = re.sub(r'\<.*\>', '', title1)
    parsDict['title'].append(title)

    # text
    body = soup.find_all('div', {'class' : 'body'})
    text0 = body[0].text
    text1 = re.sub(r'\n', ' ', text0) #Take out new lines
    text2 = re.sub(r"Source- .+","", text1) #for Times of India
    text = re.sub(r"© .+","", text2) #for allAfrica
    parsDict['text'].append(text)

    # source & date
    source = soup.find_all('div', {'class' : 'source'})
    date = re.findall(r'(?:January|February|March|April|May|June|July|August|September|October|November|Dececember)\s\d{1,2},\s\d{4}', source[0].text)
    parsDict['date'].append(date[0])
    pat2 = re.compile(r"(.*?)-", re.M)
    newssource = pat2.findall(source[0].text)
    parsDict['source'].append(newssource[0])

    meta = soup.find_all('span', {'class' : 'val'})
    # author    
    author0 = meta[0]
    author = re.sub(r'\<.*\>', '', author0.text) #Take out <>
    parsDict['author'].append(author)

    # section
    section0 = meta[1]
    section = re.sub(r'\<.*\>', '', section0.text) #Take out <>
    parsDict['section'].append(section)
    

In [106]:
pandas.DataFrame(loopThruArticles(countryDict))

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


,author,date,region,section,source,text,title
0,Jessica Elgot Political reporter,"November 13, 2017",UK,UK,"Guardian, The (London, England)",The risk posed by climate change would be fact...,Labour vows to factor climate change risk into...
1,IRIN Nairobi,"July 5, 2017",Africa,NEWS,allAfrica.com,"Everyone's heard of climate change, right? Glo...",Climate Change? What Climate Change? Nigerian ...
2,Seema Sharma,"October 18, 2017",India,Dehradun,"Times of India, The (Mumbai, India)",DEHRADUN: In a major achievement for Forest de...,9 nine govt departments to set up climate chan...
